                                                     Assignment 4 - BDAT 1002 Data Systems Architecture
                                                               Name- Abisola Olakanmi
                                                                 Student ID- 200595924
     

## Data Loading and Schema Understanding

In [1]:
#starting the session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-artists").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/17 22:16:07 INFO SparkEnv: Registering MapOutputTracker
24/04/17 22:16:07 INFO SparkEnv: Registering BlockManagerMaster
24/04/17 22:16:07 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/17 22:16:07 INFO SparkEnv: Registering OutputCommitCoordinator


In [2]:
#loading the data
path = 'gs://assignmentspark/dataset.csv'
file_type = "csv"

# CSV
infer_schema ='true'  #let computer find the data type
first_row_is_header = 'true'
delimiter =','


#import CSV
df = spark.read.format(file_type)\
    .option('inferSchema',infer_schema)\
    .option('header', first_row_is_header)\
    .option('sep', delimiter)\
    .load(path)


In [3]:
#print our columns and data types
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- track_genre: string (nullable = true)



In [4]:
#grabs the first row of the dataset
df.head()

Row(_c0=0, track_id='5SuOikwiRyPMVoIQDJUgSV', artists='Gen Hoshino', album_name='Comedy', track_name='Comedy', popularity='73', duration_ms='230666', explicit='False', danceability='0.676', energy='0.461', key='1', loudness='-6.746', mode='0', speechiness='0.143', acousticness='0.0322', instrumentalness=1.01e-06, liveness='0.358', valence='0.715', tempo=87.917, time_signature=4.0, track_genre='acoustic')

## Data Aggregation

In [5]:
#import average function
from pyspark.sql.functions import avg

In [6]:
#Question 2 Data Aggregation

#Calculating average danceability, energy, tempo, and popularity by artist

data_avg = df.groupBy("artists") \
    .agg(avg("danceability").alias("avg_danceability"),
         avg("energy").alias("avg_energy"),
         avg("tempo").alias("avg_tempo"),
         avg("popularity").alias("avg_popularity"))

# Display  results
data_avg.show()

+--------------------+------------------+-------------------+------------------+------------------+
|             artists|  avg_danceability|         avg_energy|         avg_tempo|    avg_popularity|
+--------------------+------------------+-------------------+------------------+------------------+
|       Baobinga;I.D.|             0.666|              0.966|           136.991|              10.0|
| Robert Owens;Atjazz|              NULL|               NULL|            0.0324|              NULL|
|Johann Sebastian ...|            0.0769|             0.0846|           176.329|               2.0|
|     Little Big Town|             0.459|0.48285714285714293|143.53457142857144|10.142857142857142|
|   Avicii;Aloe Blacc|             0.802|              0.645|           100.001|              33.0|
|Lucas Estrada;Twa...|             0.614|              0.802|            98.008|              51.0|
| ARMNHMR;Bella Renee|             0.405|              0.646|           141.985|              46.0|


In [7]:
# Step 2b: Identify top 5 artists with highest average track popularity

top_artists = data_avg.orderBy("avg_popularity", ascending=False).limit(5)

# Display result
top_artists.show()

+--------------------+----------------+----------+---------+--------------+
|             artists|avg_danceability|avg_energy|avg_tempo|avg_popularity|
+--------------------+----------------+----------+---------+--------------+
|Sam Smith;Kim Petras|           0.714|     0.472|  131.121|         100.0|
|    Bizarrap;Quevedo|           0.621|     0.782|  128.033|          99.0|
|       Manuel Turizo|           0.835|     0.679|   124.98|          98.0|
|Bad Bunny;Chencho...|           0.911|     0.712|   92.005|          97.0|
|Bad Bunny;Bomba E...|           0.647|     0.686|   79.928|          94.5|
+--------------------+----------------+----------+---------+--------------+



## Data Transformation

In [8]:
#import dependencies column & avg
from pyspark.sql.functions import when, col, avg

In [9]:
# Step 3: Create a new column called energy_level and defines energy levels 

df_with_energy_level = df.withColumn("energy_level", when(col("energy") > 0.8, "High Energy").otherwise("Regular Energy"))
df_with_energy_level.show()

+---+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+--------------+
|_c0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|  energy_level|
+---+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+--------------+
|  0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   False|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         

In [10]:
# Step 3b: Group by energy level and calculate the avg population and loudness for each energy_level
energy_level_agg = df_with_energy_level.groupBy("energy_level") \
    .agg(avg("popularity").alias("avg_popularity"),
         avg("loudness").alias("avg_loudness"))

# Display the results
energy_level_agg.show()

+--------------+------------------+------------------+
|  energy_level|    avg_popularity|      avg_loudness|
+--------------+------------------+------------------+
|   High Energy|31.786889141167464|-5.121428733599087|
|Regular Energy|33.990498624800956| 2.625479753796737|
+--------------+------------------+------------------+



## Data Exporting

In [11]:
# Step 4

# Filter data as 'High Energy'
high_energy_data = df_with_energy_level.filter(df_with_energy_level["energy_level"] == "High Energy")

#Export data to csv

high_energy_data.write.csv('gs://assignmentspark/high_energy_tracks.csv', header=True)
